In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset

!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-03-12 22:41:41--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  1.81MB/s    in 0.6s    

2024-03-12 22:41:42 (1.81 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [59]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [60]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [61]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [62]:
chars = sorted(list(set(text))) # 알파벳, 숫자, 특수문자 등등
vocab_size = len(chars)
print(''.join(chars))   
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [63]:
# 입력값을 토큰화 
# 일반적으로 사람들은 sub word encoding을 사용하지만, 단순하게 유지하기 위해 우리가 사용하는 것은 문자 레벨 수준의 토크나이저. 

stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)} 

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# 두 개의 정수를 임의의 문자열로 변환하고 다시 변환하는 것과 같음

print(encode('hii there'))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [64]:
# encode된 텍스트를 파이토치 텐서로 변환합니다.
# 아래의 정수 뭉치들은 위에서 input.txt의 [:1000]과 동일한 텍스트를 나타냅니다.

import torch 

data = torch.tensor(encode(text), dtype=torch.long)

print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [65]:
# 훈련데이터와 검증 데이터를 분할합시다.

n = int(0.9*len(data)) # 90%를 훈련데이터로 사용
train_data = data[:n]
val_data = data[n:] 

In [66]:
# transformer를 훈련할 때 모든 텍스트를 넣으면 비용이 많이 들기때문에, chunk 단위로 덩어리를 나눠서 사용합니다.

block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58]) 이 값들을 우리가 Transformer에 연결할 때, 이것이 말하는 것은 실제로 동시에 훈련하여 현재의 모든 위치에서 예측을 할 것이라는 것입니다.
9개의 문자 덩어리에는 실제로 8개의 개별 예시가 포함되어 있습니다. 

위의 예시에서 47의 맥락에서 보았을 때 18의 다음에 올 가능성이 높고, 18 47 56의 맥락에서 다음으로 57이 올 수 있다는 것을 알 수 있습니다.  

In [67]:
# x는 Transformer에 대한 입력이며 첫 번째 블록 크기 문자가 될 것입니다. 
# y는 다음 블록 크기가 됩니다. 문자이므로 1만큼 오프셋됩니다. 
# 하나의 컨텍스트부터 블록 크기의 컨텍스트까지를 사용하여 여기에 있는 8개의 예제 모두에 대해 훈련합니다.
# Transformer는 하나의 컨텍스트를 사용하여 다음 문자를 예측하고, 그 예측을 사용하여 다음 문자를 예측합니다. 이렇게 하면 블록 크기의 모든 것을 예측할 수 있고 블록 크기 이후에는 자르기를 시작합니다.
# 왜냐하면 Transformer는 다음 문자를 예측할 때 블록 크기 이상의 입력을 받지 않기 때문입니다. 

x = train_data[:block_size]
y = train_data[1:block_size+1]  

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target is {target}")

when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [68]:
torch.manual_seed(1337)

batch_size = 4 # transformer의 모든 순방향 역방향 패스를 처리하는 독립 시퀀스의 수 
block_size = 8 # 예측을 위한 최대 컨텍스트의 길이

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,) ) # 무작위로 선택된 인덱스의 배열 생성 
    # 0~len(data) - block_size 까지의 범위에서 정수 무작위 선택. 
    #(batch_size,)는 함수에게 생성할 무작위 정수의 개수를 알려줍니다.
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    # 결과적으로, 이 함수는 0부터 len(data) - block_size 범위 내에서 무작위로 batch_size 개수만큼의 정수를 선택
    return x, y

xb, yb = get_batch('train')

print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print("---------------------------------")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target is {target}")


inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---------------------------------
when input is [24] the target is 43
when input is [24, 43] the target is 58
when input is [24, 43, 58] the target is 5
when input is [24, 43, 58, 5] the target is 57
when input is [24, 43, 58, 5, 57] the target is 1
when input is [24, 43, 58, 5, 57, 1] the target is 46
when input is [24, 43, 58, 5, 57, 1, 46] the target is 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target is 39
when input is [44] the target is 53
when input is [44, 53] the target is 56
when input is [44, 53, 56] the target is 1
when input is [44, 53, 56, 1] the target is 58
when input is [44

In [69]:
# 간단한 신경망 

import torch 
import torch.nn as nn
from torch.nn import functional as F 
torch.manual_seed(1337) 

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # vocab_size x vocab_size (어휘의 크기 사용)
        # idx를 전달할 때 입력의 모든 단일 정수가 이 임베딩 테이블을 참조하고 뽑을 것임.
        # 인덱스에 해당하는 임베딩 테이블의 행을 꺼내므로 여기에서는 24를 임베딩 테이블로 이동하여 24번째 행을 뽑고 43번째 행을 차단

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of intergers
        logits = self.token_embedding_table(idx) #(B,T,C) (배치, 텐서, 채널) ex(4,8,65)
        # 여기서 logits은 신경망 모델이 생성한 예측값임. BigramLanguageModel을 통해 얻어진 각 단어에 대응하는 다음 단어의 로짓값을 나타냄.
        # logit이란 확률을 로그 오즈로 변환한 값으로, 신경망이 특정 클래스에 속할 원시 예측 확률을 나타내기 전의 값입니다. 

        if targets is None:
            loss = None
        else:
            # F.cross entropy 손실 함수에 적합한 형태로 만든다.
            B, T , C = logits.shape
            logits = logits.view(B*T, C) # (B*T, C) (32, 65)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits , loss

    def generate(self, idx, max_new_tokens):
        # idx는 (B,T) 사이즈의 정수 텐서입니다.
        for _ in range(max_new_tokens):
            # 예측값을 얻기 위해 마지막 토큰을 사용합니다.
            logits, loss = self(idx)
            # 마지막 타임 스텝에 집중합니다.
            logits = logits[:,-1,:] # (B,C)가 됩니다.
            # 로짓을 소프트맥스로 변환합니다.
            probs = F.softmax(logits, dim=-1) # (B,C)가 됩니다.
            # distribution에서 샘플링합니다.
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # 샘플링된 idx_next를 idx에 추가합니다.
            idx = torch.cat((idx, idx_next), dim=-1)  # (B, T+1)
        
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

# idx를 1x1 크기의 텐서로 0으로 초기화되며, 데이터 타입은 정수임. 언어 모델에서는 주로 시작 토큰을 나타내는데 사용한다.
print(decode(m.generate( idx = torch.zeros((1,1), dtype=torch.long) , max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [70]:
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)

In [71]:
batch_size = 32 # 배치 사이즈 32

for steps in range(10000):

    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)    
    optimizer.zero_grad(set_to_none=True)   
    loss.backward()
    optimizer.step()

print(loss.item())

2.572469472885132


In [72]:
print(decode(m.generate( idx = torch.zeros((1,1), dtype=torch.long) , max_new_tokens=500)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercckehathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht anjx?

DUThineent.

Lavinde.
athave l.
KEONGBUCHandspo be y,-hedarwnoddy scace, tridesar, wne'shenous s ls, theresseys
PlorseelapinghiybHen yof GLUCEN t l-t E:
I hisgothers w dere! ABer wotouciullle's


기존의 예측값 : Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3 

학습 후 예측값 (loss 2.5) : 
PENund rron in tenghe;

CROUClereave t omy is.
KISThos, bucesome, l.
ENENRER:
Wiero
DUSe, ysoupad it

### The mathematical trick in self attention

In [73]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch time channel
x = torch.randn(B, T, C) # (4, 8, 2)

x.shape

torch.Size([4, 8, 2])

In [74]:
x

tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])

In [75]:
# 해당 시퀀스의 모든 토큰에 대해 독립적으로 모든 단일 

xbow = torch.zeros((B,T,C))

for b in range(B):  # 배치 순회 
    for t in range(T): # 시퀀스 내 위치 순회
        xprev = x[b, :t+1] # (t, C)      # 현재 위치 t와 그 이전 위치의 모든 토큰을 선택. 여기서 x[b:t+1]은 배치의 b번째 항목에서 처음부터 t+1까지의 토큰 선택하여 xprev에 저장
        xbow[b,t] = torch.mean(xprev, 0) # 선택된 텐서에 대해 0번째 차원에 대한 평균 계산. 각 채널에 대해 현재 시간 단계까지의 모든 값의 평균을 계산함. 

# 결론 : xbow는 시퀀스의 각 단계에서 이전 단계들의 정보를 종합적으로 반영하는 새로운 방식

In [76]:
# version2
wei = torch.tril(torch.ones(T,T)) 
wei = wei / wei.sum(1, keepdim=True)    
xbow2 = wei @ x # (B, T, T) @ (B, T, C) -> (B, T, C)    

torch.allclose(xbow, xbow2)

False

In [77]:
xbow[0] 

# 아래의 결과에서 첫번째 행은 그대로임. 
# 두번째는 첫번째 행과 두번째 행의 평균 , 세번째는 첫번째, 두번째, 세번째 행의 평균, 네번째는 첫번째, 두번째, 세번째, 네번째 행의 평균을 나타냄.
# 마지막 행은 모든 토큰의 평균임. 
# 하지만 비효율적임

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [78]:
torch.tril(torch.ones((3,3)))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [79]:
# 효율적인 방법 (행렬)

torch.manual_seed(42)

a = torch.tril(torch.ones((3,3)))
a = a / torch.sum(a,1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b 

print('a = ')
print(a)
print('--')
print('b= ')
print(b)
print('--')
print('c = ')
print(c)    

a = 
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b= 
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c = 
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [82]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [87]:
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril ==0 , float('-inf'))
# wei = F.softmax(wei, dim=1)
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [80]:
#version3

tril = torch.tril(torch.ones(T,T)) # 하삼각행렬
wei = torch.zeros((T,T)) # (8,8)
wei = wei.masked_fill(tril ==0 , float('-inf')) # 마스크를 사용하여 하삼각행렬의 0을 -inf로 채움   
wei = F.softmax(wei, dim=1) # (8,8)
xbow3 = wei @ x # (B, T, T) @ (B, T, C) -> (B, T, C)
torch.allclose(xbow, xbow3)

False

In [ ]:
xbow[0] , xbow2[0]

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))